In [ ]:
import pandas as pd
import numpy as np
import nbimporter
import util_func

class Abaqus:
    def __init__(self, job_name, job_directory, data_directory, BC_file = False):
        # Initialize class variables with file name strings
        self.job = job_name
        self.job_path = r"{folder}\{job}".format(folder=job_directory,job=self.job)
        self.data_path = r"{folder}\{job}".format(folder=data_directory,job=self.job)

        # Read Abaqus input file for model
        inp_file = r"{path}.inp".format(path = self.job_path)
        with open(inp_file, "r") as file:
            inp = [line.rstrip() for line in file]
            file.seek(0)
            self.lines_inp = file.readlines()
            file.close()

        # Extract coordinate strings from input file
        coord_str = tuple(i.split(',') for i in
                          inp[(inp.index('*Node')+1):inp.index('*Element, type=C3D8R')])
        
        # Create class variables storing coordinates, node count, and dof count info
        self.coord     = np.array(coord_str, dtype=float)[:,1:]
        self.num_nodes = len(self.coord)
        self.num_dof   = 3*self.num_nodes

        # Extract connectivity matrix strings from input file
        connect_str = tuple(i.split(',') for i in
                            inp[inp.index('*Element, type=C3D8R')+1:inp.index('*Nset, nset=Bottom')])
        
        # Create class variables storing connectivity matrix and element count
        self.connect = np.array(connect_str, dtype=int)[:,1:] - 1
        self.num_el  = len(self.connect)
        
        # Declare default flag indicating boundary condition array has not yet been constructed
        self.BC_flag = False
        # Read in boolean array denoting boundary condition indices from file if it exists
        if BC_file:
            self.BC_mask = pd.read_csv(BC_file).to_numpy(dtype=bool)[:,1]
            self.BC_flag = True # switch BC_flag to true once boundary condition array has been read in
        
        # Initialize list of tags common to all results files
        common_tags = ["beta_d","beta_m","beta_p","t"]
        
        # Initialize tags for use in specific data storage files
        dof_tags    = [["u{node}".format(node=i), "v{node}".format(node=i), "w{node}".format(node=i)] 
                        for i in range(1,self.num_nodes+1)]
        strain_tags = [["E22_{node}".format(node=i), "E33_{node}".format(node=i)] for i in range(1,self.num_nodes+1)]
        beta_tags   = [["beta1_{node}".format(node=i), "beta2_{node}".format(node=i)] for i in range(1,self.num_nodes+1)]
        alpha_tags  = [["Ei_{node}".format(node=i)] for i in range(1,self.num_nodes+1)]
        dam_tags    = [["dam_{node}".format(node=i)] for i in range(1,self.num_nodes+1)]
        del_tags    = [["status_{el}".format(el=i)] for i in range(1,self.num_el+1)]
        
        # Combine storage tags into single list for handling with comprehension
        tags_list = [dof_tags, strain_tags, beta_tags, alpha_tags, dam_tags, del_tags]

        # Create class variable with common tags and file specific storage tags combined as tuple elements
        self.data_tags = tuple(common_tags + [x for xs in tags for x in xs] for tags in tags_list)
        
        # Create class variable containing tags for the names of the storage files themselves
        self.file_tags = ("Snapshots", "Strain", "Beta", "E_Alpha", "Damage", "Status")
        
        
    def get_stiffness(self,mu): # Function to retrieve stiffness matrices from files
        # Initialize storage for stiffness matrix
        Kn = np.zeros((self.num_dof, self.num_dof)) 
        
        # Read in stiffness matrix from file
        stif_file = r"{path}_STIF2.mtx".format(path = self.job_path)
        with open(stif_file, "r") as file:
            stif = [line.strip().split() for line in file]
            file.close()

        # Insert non-zero stiffness matrix elements at designated indices
        for line in stif:
            i, j = int(line[0])-1, int(line[1])-1
            val = float(line[2])
            Kn[i,j] = val

        # Use stiffness matrix to determine indices of boundary conditions and store as BC_mask if not 
        # already retrieved from file 
        if not self.BC_flag:
            BC_ind = [i for i in range(self.num_dof) if np.diag(Kn)[i] > 10**30]
            mask = np.ones(len(Kn), dtype=bool)
            mask[BC_ind] = False
            
            self.BC_mask = mask
            
            mask_frame = pd.DataFrame(self.BC_mask)
            mask_frame.to_csv(r"{path}_BC_mask.csv".format(path=self.data_path))
            self.BC_flag = True
        
        # Reduce stiffness matrix by removing rows and columns associated with boundary conditions to save memory
        Kc_n = Kn[self.BC_mask, :]
        Kc_n = Kc_n[:, self.BC_mask]
        Kc_n = pd.DataFrame(Kc_n)
        
        # Write stiffness matrix to file
        Kc_n.to_csv(r"{path}_Kc_{beta_d}_{beta_m}_{beta_p}.csv"
                    .format(path=self.data_path, beta_d=mu[0], beta_m=mu[1], beta_p=mu[2]))
        
        # CSV file corruption is somewhat common with matrices of this size.
        # Run a check on the CSV to ensure it can be read, and rewrite the CSV if it cannot.
        corrupt = True
        while corrupt:
            try:
                check = pd.read_csv(r"{path}_Kc_{beta_d}_{beta_m}_{beta_p}.csv"
                    .format(path = self.data_path, beta_d=mu[0], beta_m=mu[1], beta_p=mu[2]), index_col=0)
                corrupt = False
            except:
                Kc_0.to_csv(r"{path}_Kc_{beta_d}_{beta_m}_{beta_p}.csv"
                    .format(path = self.data_path, beta_d=mu[0], beta_m=mu[1], beta_p=mu[2]))
        
        
    def get_max_load(self, mu): # Function to retrieve FE load vector
        
        # This function requires the BC mask but cannot create it internally.
        # Ensure it exists before proceeding.
        assert self.BC_flag, """The boundary condition mask has not been defined. 
                                Retrieve it via get_stiffness() or directly from 
                                BC_file during __init__."""
        
        # Read in load vector from file
        load_file = r"{path}_LOAD2.mtx".format(path = self.job_path)
        with open(load_file, "r") as file:
            load = [line.strip().split() for line in file]
            file.close()

        # Intialize load vector storage array
        Fn = np.zeros(self.num_dof)

        # Insert non-zero load vector elements at designated indices
        for line in load[2:]:
            i = int(line[0])-1
            val = float(line[1])
            Fn[i] = val

        # Reduce load vector by removing elements at boundary condition indices
        Fc_n = pd.DataFrame(Fn[self.BC_mask])
        
        # Write load vector to file
        Fc_n.to_csv(r"{path}_Fc_{beta_d}_{beta_m}_{beta_p}.csv"
                    .format(path=self.data_path, beta_d=mu[0], beta_m=mu[1], beta_p=mu[2]))
    
    
    def init_data_storage(self):# Function for clearing and initializing data storage files
        
        # Create empty data storage CSVs with appropriate column labels
        for j in range(len(self.data_tags)):
            pd.DataFrame(columns=self.data_tags[j]).to_csv(r"{path}_{tag}.csv"
                              .format(path=self.data_path, tag=self.file_tags[j]), mode=
                                     'w')
            
#         # Create empty data storage text file for tear results
#         tear_file = r"{path}_tear_results.txt".format(path=self.data_path)
#         with open(tear_file, "w+") as file:
#             file.close()
        
    def get_data(self, mu): # Funtion for retrieval of FE results data
        
        # This function requires the BC mask but cannot create it internally.
        # Ensure it exists before proceeding.
        assert self.BC_flag, """The boundary condition mask has not been defined. 
                                Retrieve it via get_stiffness() or directly from 
                                BC_file during __init__."""
        
        # Initialize values for data labeling by mu parameter case
        mu_arr = np.array(mu)
        
        # Read in status file
        sta_file = r"{path}.sta".format(path = self.job_path)
        with open(sta_file, "r") as file:
            sta = [line.strip().split() for line in file]
            file.close()

        # Save data regarding element deletion to dataframe
        status = pd.DataFrame(sta[5:-2])
        
        # Use step increments from status file to calculate time step values and store in array
        step_inc = np.array(tuple(status[8].loc[status[0]=='1'].loc[
                                    ~status[2].str.contains('U')].values), dtype=float)
        step_times = np.array([sum(step_inc[0:n]) for n in range(1,len(step_inc)+1)])
        
        
        # Read .DAT to retrieve all FE results data
        dat_file = r"{path}.dat".format(path = self.job_path)
        with open(dat_file, "r") as file:
            dat = [line.strip() for line in file]
            file.close()
        
        # Retrieve starting and ending rows for displacement snapshots 
        U_start = np.array([i for i, x in enumerate(dat) if x == 'NODE FOOT-  U1             U2             U3'])
        U_end = np.array([i for i, x in enumerate(dat) if 
                          (x.startswith('AT NODE') and dat[i-1].startswith('MAXIMUM'))])
        U_start = U_start+3
        U_end = U_end-2
        
        # Retrieve starting and ending rows for nodal data (state-dependent variables)
        node_start = np.array([i for i, x in enumerate(dat) if x.startswith('NODE  FOOT-   SDV2')])
        node_end = np.array([i for i, x in enumerate(dat) if 
                          (x.startswith('NODE') and dat[i-1].startswith('MAXIMUM'))])
        node_start = node_start+3
        node_end = node_end-2
        
        # Retrieve starting and ending rows for status of deleted elements
        del_start = np.array([i for i, x in enumerate(dat) if x == 'ELEMENT  PT FOOT-       SDV16'])
        del_end = np.array([i for i, x in enumerate(dat) if 
                          (x.startswith('ELEMENT') and dat[i-1].startswith('MAXIMUM'))])
        del_start = del_start + 3
        del_end = del_end - 2

        # Initialize list of empty dataframes with column tags from class initialization in which to store results data
        FE_data = [pd.DataFrame(columns=tags) for tags in self.data_tags]
        
        # Initialize a storage vector to be used for displacement snapshots
        U = np.zeros((self.num_dof,))
        
        # Iterate over time steps and retrieve relevant data at each snapshot
        for i in range(len(step_times)):
            # Gather .DAT file slices for displacement, nodal data, and element data for current time step
            tempU = dat[U_start[i]:U_end[i]]
            tempN = dat[node_start[i]:node_end[i]]
            tempE = dat[del_start[i]:del_end[i]]
            
            # Collect displacement, nodal, and element data into numpy arrays
            Uc = np.array([x.split()[1:] for x in tempU], dtype=float).flatten()
            sep_N = np.array([x.split()[1:] for x in tempN], dtype=float)
            sep_E = np.array([x.split() for x in tempE])

            # Map reduced displacement onto full displacement via BC_mask indices 
            U[self.BC_mask] = Uc
            
            # Separate nodal data into its components
            E_hoop_axial = sep_N[:,:2].flatten()
            Beta         = sep_N[:,2:4].flatten()
            Alpha        = sep_N[:,4].flatten()
            Dam          = sep_N[:,5].flatten()
            
            # Separate element dat into index and element deletion status arrays
            stat_ind = np.array(sep_E[:,0], dtype=int) - 1
            Status           = np.zeros((self.num_el,))
            Status[stat_ind] = np.array(sep_E[:,2], dtype=float).flatten()
            
            # Place data arrays in a list for iteration
            data_list = [U, E_hoop_axial, Beta, Alpha, Dam, Status]
            
            # Iterate over list of arrays and append data to corresponding dataframes
            for j in range(len(FE_data)):
                FE_data[j].loc[len(FE_data[j])] = np.hstack((mu_arr,step_times[i],data_list[j]))
        
        # Iterate over list of file tags and write each corresponding dataframe to file
        for j in range(len(self.file_tags)):
            FE_data[j].to_csv(r"{path}_{tag}.csv"
                              .format(path=self.data_path, tag=self.file_tags[j]), mode='a', header=False)


In [ ]:
#         init_file = r"{path}_INIT.inp".format(path = self.job_path)
#         with open(init_file, "r") as file:
#             self.lines_init = file.readlines()
#             file.close()
            
#         ###############################################################################
        
#         K0 = np.zeros((self.num_dof, self.num_dof))
        
#         init_stif_file = r"{path}_INIT_STIF2.mtx".format(path = self.job_path)

#         with open(init_stif_file, "r") as file:
#             stif = [line.strip().split() for line in file]
#             file.close()

#         for line in stif:
#             i, j = int(line[0])-1, int(line[1])-1
#             val = float(line[2])
#             K0[i,j] = val
            
#         Kc_0 = K0[self.BC_mask, :]
#         Kc_0 = Kc_0[:, self.BC_mask]
#         Kc_0 = pd.DataFrame(Kc_0)
        
#         Kc_0.to_csv(r"{path}_Kc0_{beta_d}_{beta_m}_{beta_p}.csv"
#                     .format(path = self.data_path, beta_d=mu[0], beta_m=mu[1], beta_p=mu[2]))
        
#         corrupt = True
#         while corrupt:
#             try:
#                 check = pd.read_csv(r"{path}_Kc0_{beta_d}_{beta_m}_{beta_p}.csv"
#                     .format(path = self.data_path, beta_d=mu[0], beta_m=mu[1], beta_p=mu[2]), index_col=0)
#                 corrupt = False
#             except:
#                 Kc_0.to_csv(r"{path}_Kc0_{beta_d}_{beta_m}_{beta_p}.csv"
#                     .format(path = self.data_path, beta_d=mu[0], beta_m=mu[1], beta_p=mu[2]))

In [ ]:
#     def get_tear_len(self, mu): # Function for saving final tear results data
        
#         # Create file path string for tear data storage
#         tear_log = self.job_path.replace(self.job, "tear_log.txt")

#         # Read length of tear propagation in either direction (up or down)
#         da_i, da_c = [float(x) for x in util_func.read_n_to_last_line(tear_log, n=5).split()]
        
#         # Compute total tear length at end of simulation
#         tear_len = 3.1 + da_i + da_c
        
#         # Create list with beta parameters and tear length data
#         tear_data = [mu[0], mu[1], mu[2], da_i, da_c, tear_len, '\n']
        
#         # Write tear length data to file
#         tear_file = r"{path}_tear_results.txt".format(path=self.data_path)
#         with open(tear_file, "a") as file:
#             file.write(' '.join(str(i) for i in tear_data))
#             file.close()